In [4]:
import numpy as np
from scipy.spatial.distance import cdist
from scipy.optimize import minimize
import requests

def get_pos_from_url(address='http://doye.chem.ox.ac.uk/jon/structures/LJ/points/', N=7):
    url_address = address + str(N)
    data_str = requests.get(url_address).text
    return parse_url_text(data_str)    
    
def parse_url_text(data_str):
    x_array = []
    text = data_str.split('\n')
    for line in text:
        [x_array.append(float(i)) for i in line.split()]
    return np.array(x_array)

def dist(N_atoms):
    pos = get_pos_from_url(N = N_atoms)
    pos = np.reshape(pos,[N_atoms,3])
    #pos = 10*np.random.random(N_atoms*3)
    #pos = np.reshape(pos,[N_atoms,3])
    distance = cdist(pos,pos,'euclidean')
    iu1 = np.triu_indices(N_atoms)
    distance = distance[iu1]
    index = np.argwhere(distance==0)
    distance = np.delete(distance,index)
    return distance
    
def LJ(r):
    r6 = np.power(r,6)
    r12 = np.multiply(r6,r6)
    return 4*(1/r12 - 1/r6)
    
def Total_Energy(positions):

    E = LJ(positions)
    Energy = np.sum(E)
    return Energy

N_attempts = 50
f_values = []
x_values = []

#First Scipy optimize method
def ground_state_potential(N_atom):
    pos = dist(N_atom)
    for i in range(N_attempts):
        
        res = minimize(Total_Energy, pos, method='CG', tol=1e-4)
        f_values.append(res.fun)
        x_values.append(res.x)
        if i%10==0:
           print('Step: ', i, '  Value:', res.fun, " Number of Iterations: ", res.nit)

    print('The ground state potential is:  ', min(f_values))

#Choosing new method of Nelder-Mead to compare results

def ground_state_potential2(N_atom):
    pos = dist(N_atom)
    for i in range(N_attempts):
        
        res = minimize(Total_Energy, pos, method='Nelder-Mead', tol=1e-4)
        f_values.append(res.fun)
        x_values.append(res.x)
        if i%10==0:
           print('Step: ', i, '  Value:', res.fun, "Number of Iterations: ", res.nit)

    print('The ground state potential is:  ', min(f_values))
    
#Choosing another method of Powell to compare results

def ground_state_potential3(N_atom):
    pos = dist(N_atom)
    for i in range(N_attempts):
        
        res = minimize(Total_Energy, pos, method='Powell', tol=1e-4)
        f_values.append(res.fun)
        x_values.append(res.x)
        if i%10==0:
           print('Step: ', i, '  Value:', res.fun, "Number of Iterations: ", res.nit)

    print('The ground state potential is:  ', min(f_values))
    
#Choosing the fourth method of BFGS to compare results
def ground_state_potential4(N_atom):
    pos = dist(N_atom)
    for i in range(N_attempts):
        
        res = minimize(Total_Energy, pos, method='BFGS', tol=1e-4)
        f_values.append(res.fun)
        x_values.append(res.x)
        if i%10==0:
           print('Step: ', i, '  Value:', res.fun, "Number of Iterations: ", res.nit)

    print('The ground state potential is:  ', min(f_values))

In [3]:
print("Results for Scipy Minimization Method Using CG:", "\n")
ground_state_potential(20)

Results for Scipy Minimization Method Using CG: 

Step:  0   Value: -189.99999999991604  Number of Iterations:  56
Step:  10   Value: -189.99999999991604  Number of Iterations:  56
Step:  20   Value: -189.99999999991604  Number of Iterations:  56
Step:  30   Value: -189.99999999991604  Number of Iterations:  56
Step:  40   Value: -189.99999999991604  Number of Iterations:  56
The ground state potential is:   -189.99999999991604
